In [1]:
#Libraries Used
import sqlite3
from tabulate import tabulate
#User-Defined Function:Display the ledgers stored as database
def showtab(db):
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    print(tabulate(cursor.fetchall(), headers = ["The Ledgers are"]))
    conn.close()
    
#User-Defined function creating the ledger
def create_table(name, db):
    conn = sqlite3.connect(db)
    cuobj = conn.cursor()
    st = f"""CREATE TABLE IF NOT EXISTS {name} (
        Sl_No INTEGER PRIMARY KEY,
        Dr_Date TEXT,
        Dr_Particular TEXT,
        Dr_JF INTEGER,
        Dr_Amount INTEGER,
        Cr_Date TEXT,
        Cr_Particular TEXT,
        Cr_JF INTEGER,
        Cr_Amount INTEGER
    );"""
    cuobj.execute(st)
    conn.commit()
    conn.close()
#input values from the user
def input_val():
    Date = []
    li = ['Enter Year ','Enter Month ','Enter Day ']
    i = 0
    while(i <= 2):
        dat = input(li[i])
        print()
        if(i == 0 and len(dat) == 4):
            Date.append(dat)
            i += 1
        elif(i == 1 and len(dat) == 2):
            Date.append(dat)
            i += 1
        elif(i == 2 and len(dat) == 2):
            Date.append(dat)
            i += 1
        else:
            print("Enter Date of the format YYYY-MM-DD")
        print()
    date = str(Date[0])+'-'+str(Date[1])+'-'+str(Date[2])
    Part = input("Enter Particulars ")
    JF = input("Enter Journal Folio ")
    Amt = input("Enter Amount ")
    
    return date,Part,JF,Amt

 #adding records to the existing ledger   
def update_end(name, db, sno=0):
    conn = sqlite3.connect(db)
    cuobj = conn.cursor()

    if sno == 0:
        st = "Select * from "+str(name)+' ;'
        cuobj.execute(st)
        Data = cuobj.fetchall() 
        sno = len(Data) + 1
    

    while True:
        c = input("Credit or Debit...\nEnter Cr/Dr ")
        if c.lower() == "dr":
            Date, Part, JF, Amt = input_val()
            st = """INSERT INTO {0} (Sl_No, Dr_Date, Dr_Particular, Dr_JF, Dr_Amount)
                      VALUES (?,?,?,?,?);""".format(str(name))
            cuobj.execute(st,(str(sno),str(Date),str(Part),str(JF),str(Amt)))
            break
        elif c.lower() == "cr":
            Date, Part, JF, Amt = input_val()
            st = """INSERT INTO {0} (Sl_No, Cr_Date, Cr_Particular, Cr_JF, Cr_Amount)
                      VALUES (?,?,?,?,?);""".format(str(name))
            cuobj.execute(st,(str(sno),str(Date),str(Part),str(JF),str(Amt)))
            break
        else:
            print("Invalid input")
    
    conn.commit()
    conn.close()
#Displaying the ledger
def diplay(name,db):
    conn = sqlite3.connect(db)
    cuobj = conn.cursor()
    st = "Select * from "+str(name)+' ;'
    cuobj.execute(st)
    Data = cuobj.fetchall()
    table = tabulate(Data,headers = ["Sl_No", "Dr_Date", "Dr_Particular", "Dr_JF", "Dr_Amount", "Cr_Date", "Cr_Particular", "Cr_JF", "Cr_Amount"])
    print(table)
    conn.commit()
    conn.close()
#Adding records between the existing ledgers
def update_between(name, db):
    conn = sqlite3.connect(db)
    cuobj = conn.cursor()
    sno = int(input("Enter Sl No. to be inserted in "))
    st = "Select * from "+str(name)+' ;'
    cuobj.execute(st)
    Data = cuobj.fetchall()
    if(sno >= len(Data) or len(Data) == 0):          #Error Handling: If user eneters the sl. No. = last entry or there are no prior entries
        print("Please select option 2")
    else:    
        st = "UPDATE {0} SET Sl_No = Sl_No + 1 WHERE Sl_No > {1} ORDER BY Sl_No ASC;".format(name, sno)
        cuobj.execute(st)
        update_end(name, db, sno)
    
    conn.commit()
    conn.close()
#Deleting the whole ledger
def del_tab(name,db):
    conn = sqlite3.connect(db)
    cuobj = conn.cursor()
    st = "DROP TABLE [IF EXISTS] "+db+"."+name+";"
    conn.execute(st)
    conn.commit()
    conn.close()
#Deleting a specific record in the ledger  
def del_rec(name,db,sno):
    conn = sqlite3.connect(db)
    cuobj = conn.cursor()
    st = f"""DELETE FROM {name}
    WHERE Sl_No = (?);"""
    conn.execute(st,(sno))
    conn.commit()
    conn.close()
#Updating the ledger from the records    
def update_rec(name,db,sno):
    del_rec(name,db,sno)
    update_end(name,db,sno)
    

In [ ]:
#MAIN
txt = "    *******WELCOME TO TRACKEDGE*******    "

x = txt.center(20)

print(x)

while (True):
    db = input("Enter Account Number ")
    print()
    showtab(db)
    print()
    c = 0
    while(c != 4):
        c = int(input('''Commands:
        1. Create New Ledger
        2. Access Existing Ledger
        3. Delete Ledger
        4. Exit
        Enter 1/2/3/4
        '''))
        print()
        if(c == 1):
            name = input("Enter Ledger name")
            create_table(name, db)
            diplay(name,db)
        elif(c == 2):
            name = input("Enter Ledger name")
            diplay(name,db)
        elif(c == 3):
            name = input("Enter Ledger name")
            del_tab(name,db)
        elif(c != 4):
            print("Invalid Command")
        
        print()
        
        co = 0
        while(co != 5):
            co = int(input('''Commands:
            1. New Ledger Entry
            2. Update Ledger in between
            3. Update Existing Record
            4. Delete Existing Record
            5. Exit
            Enter 1/2/3/4/5
            '''))
            print()
            if(co == 1):
                update_end(name,db)
                diplay(name,db)
            elif(co == 2):
                update_between(name, db)
                diplay(name,db)
            elif(co == 3):
                sno = input("Enter Sl. No. of record to be updated")
                update_rec(name,db,sno)
                diplay(name,db)
            elif(co == 4):
                sno = input("Enter Sl. No. of record to be deleted")
                del_rec(name,db,sno)
                diplay(name,db)
            elif(co != 5):
                print("Invalid Command")
            print()
        
    com = int(input('''Commands:
    1. Access/Create New Account
    2. Exit
    Enter 1/2'''))
    
    if(com == 1):
        continue
    else:
        break
    print()
    

    *******WELCOME TO TRACKEDGE*******    
